### 🛠️ 도구2: Retrieval(검색)

Assistants API에서 또 다른 강력한 도구는 [검색](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval)이다. 

주요 기능

- 질문에 답변할 때 Assistant가 제공된 문서나 지식 기반으로 답변할 수 있게하는 기능입니다.
- 검색은 독점적인 제품 정보나 사용자가 제공한 문서 등 모델 외부의 지식으로 Assistant 의 답변을 보강합니다.
- 파일을 업로드하여 어시스턴트에 전달하면 OpenAI가 자동으로 문서를 청크 처리(분할)하고, 임베딩을 색인화 및 저장하며, 벡터 검색을 구현하여 관련 콘텐츠를 검색하여 사용자 쿼리에 답변합니다.

기존의 Assistant 에 **검색 도구(Retrieval Tools) 를 추가** 하고, **업로드한 파일을 연결** 까지 완료했다면, 이제 Assistant 를 사용하여 검색을 수행할 수 있습니다.

수행하는 방식은 기존과 동일합니다.

질문의 내용을 이해하고, Assistant 에게 질문을 제출하면 됩니다.

다만, **"파일로부터 검색해줘" 라는 명령어를 사용하여 Assistant 에게 파일로부터 검색을 수행하도록 지시** 해야 합니다.


In [1]:
import time
from openai import OpenAI

client = OpenAI()
assistant = client.beta.assistants.create(
    name="Retrieval Tool test",
    instructions="This assistant will help you retrieve files from the OpenAI API.",
    model="gpt-4o-mini",
)
files = list(client.files.list())
    
if not files:
    print("삭제할 파일이 없습니다.")


print(f"총 {len(files)}개의 파일을 삭제합니다.")

deleted_count = 0
for file in files:
    try:
        client.files.delete(file.id)
        print(f"파일이 삭제되었습니다: {file.filename} (ID: {file.id})")
        deleted_count += 1
        time.sleep(0.5)  # API 요청 간 짧은 대기 시간
    except Exception as e:
        print(f"파일 삭제 중 오류 발생: {file.filename} (ID: {file.id})")
        print(f"오류 메시지: {str(e)}")
    
    print(f"총 {deleted_count}개의 파일이 삭제되었습니다.")


총 2개의 파일을 삭제합니다.
파일이 삭제되었습니다: SPRI_AI_Brief_2023년12월호.pdf (ID: file-AalSKdWM9TleKuSbai0NfAjS)
총 1개의 파일이 삭제되었습니다.
파일이 삭제되었습니다: SPRi AI Brief_6월호_산업동향 최종.pdf (ID: file-zIEvM2e9fm5TpvwdwzRG1VD6)
총 2개의 파일이 삭제되었습니다.


In [2]:
# 파일 업로드를 위한 함수를 정의합니다.
def upload_files(files):
    uploaded_files = []
    for filepath in files:
        file = client.files.create(
            file=open(
                # 업로드할 파일의 경로를 지정합니다.
                filepath,  # 파일경로. (예시) data/sample.pdf
                "rb",
            ),
            purpose="assistants",
        )
        uploaded_files.append(file.id)
        print(f"[업로드한 파일 ID]\n{file.id}")
    return uploaded_files

In [3]:
files_to_upload = [
    "/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/data/SPRi AI Brief_6월호_산업동향 최종.pdf",
    "/Users/jiminking/Documents/김지민/projects/langchain_study/03.OpenAI/data/SPRI_AI_Brief_2023년12월호.pdf",
]

# 파일을 업로드합니다.
file_ids = upload_files(files_to_upload)
print()
print("file_ids : ", file_ids)

[업로드한 파일 ID]
file-AQULGHjWks1kQK2njsbAwjYF
[업로드한 파일 ID]
file-rjzc51lhmqvrcnyl0OOb3Unc

file_ids :  ['file-AQULGHjWks1kQK2njsbAwjYF', 'file-rjzc51lhmqvrcnyl0OOb3Unc']


In [4]:
# 업로드한 모든 파일 ID 와 파일명을 출력합니다.
for file in client.files.list():
    print(f"[파일 ID] {file.id} [파일명] {file.filename}")

[파일 ID] file-rjzc51lhmqvrcnyl0OOb3Unc [파일명] SPRI_AI_Brief_2023년12월호.pdf
[파일 ID] file-AQULGHjWks1kQK2njsbAwjYF [파일명] SPRi AI Brief_6월호_산업동향 최종.pdf


In [5]:
import json

"""
`show_json` 함수는 인자로 받은 객체의 모델을 JSON 형태로 변환하여 출력합니다.
"""
def show_json(obj):
    # obj의 모델을 JSON 형태로 변환한 후 출력합니다.
    display(json.loads(obj.model_dump_json()))
    
ASSISTANT_ID = assistant.id

In [6]:
# Assistant 의 설정을 업데이트합니다.
assistant = client.beta.assistants.update(
    ASSISTANT_ID,
    # retrieval 도구를 추가합니다.
    tools=[
        {"type": "code_interpreter"},

    ],
     tool_resources={
        "code_interpreter": {
            "file_ids": file_ids}

    },
    # Assistant 의 역할을 설명합니다.
    instructions="You are a expert in Document Retrieval. Answer questions using the uploaded documents.",

)

# 업데이트된 Assistant 정보를 출력합니다.
show_json(assistant)

{'id': 'asst_PXe29yEDZqV1tnV26wPsrC34',
 'created_at': 1722444964,
 'description': None,
 'instructions': 'You are a expert in Document Retrieval. Answer questions using the uploaded documents.',
 'metadata': {},
 'model': 'gpt-4o-mini',
 'name': 'Retrieval Tool test',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': {'file_ids': ['file-rjzc51lhmqvrcnyl0OOb3Unc',
    'file-AQULGHjWks1kQK2njsbAwjYF']},
  'file_search': None},
 'top_p': 1.0}

In [8]:
def create_new_thread():
    # 새로운 스레드를 생성합니다.
    thread = client.beta.threads.create()
    return thread

thread = create_new_thread()
# 새로운 스레드를 생성합니다.
show_json(thread)

{'id': 'thread_fx0Y0RlScTeqXG1kohrksJS3',
 'created_at': 1722445028,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

In [10]:
import time


# 반복문에서 대기하는 함수
def wait_on_run(run, thread_id):
    while run.status == "queued" or run.status == "in_progress":
        # 3-3. 실행 상태를 최신 정보로 업데이트합니다.
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


def submit_message(assistant_id, thread_id, user_message):
    # 3-1. 스레드에 종속된 메시지를 '추가' 합니다.
    client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=user_message
    )
    # 3-2. 스레드를 실행합니다.
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
    )
    return run


def get_response(thread_id):
    # 3-4. 스레드에 종속된 메시지를 '조회' 합니다.
    return client.beta.threads.messages.list(thread_id=thread_id, order="asc")


def print_message(response):
    for res in response:
        print(f"[{res.role.upper()}]\n{res.content[0].text.value}\n")


def ask(assistant_id, thread_id, user_message):
    run = submit_message(
        assistant_id,
        thread_id,
        user_message,
    )
    # 실행이 완료될 때까지 대기합니다.
    run = wait_on_run(run, thread_id)
    print_message(get_response(thread_id).data[-2:])
    return run

In [11]:
ASSISTANT_ID = assistant.id  # 업데이트된 Assistant ID를 지정합니다.
thread_id = create_new_thread().id  # 새로운 스레드를 생성합니다.

# 질문을 던집니다.
run = ask(
    ASSISTANT_ID,
    thread_id,
    "빌게이츠의 AI 에이전트에 대한 견해를 SPRI_AI_Brief_2023년12월호.pdf 파일에서 검색하여 알려주세요. "
    "200~300 단어로 자세히 설명해 주세요. 반드시 한글로 써주세요",
)

[ASSISTANT]
두 개의 파일에서 추출된 텍스트의 초기 내용이 대부분 공백인 것으로 보아, 텍스트 추출에 문제가 발생한 것으로 추정됩니다. 이 경우, PDF 파일의 구조적 요소가 문제를 일으킬 수 있습니다.

빌게이츠의 AI 에이전트에 대한 견해가 포함된 내용을 검색하기 위해, 파일들을 다시 분석하여 그들이 포함한 주요 내용을 접할 수 있도록 하겠습니다. 다시 시도해 보겠습니다.

[ASSISTANT]
두 개의 파일에서 "빌게이츠"라는 키워드와 관련된 내용을 찾을 수 없었습니다. 텍스트 추출에 문제가 있었던 것 같아, 내용을 직접 분석하는 데 한계가 있을 수 있습니다. 

만약 특정 문서를 따로 제공해 주시면, 문서의 내용을 보다 정확하게 분석할 수 있게 됩니다. 추가적인 지원이 필요하시거나 다른 파일을 업로드하시려면 말씀해 주세요. 또는 PDF 파일로부터 직접 언급하고자 하는 내용의 특정 구문을 알려주시면 그 부분을 강조하여 검색하도록 하겠습니다.



할루시네이션 발생 안함!!!!! 

In [12]:
run = ask(
    ASSISTANT_ID,
    thread_id,
    "SPRI_AI_Brief_2023년12월호.pdf 파일을 요약해주세요. ")
   

[USER]
SPRI_AI_Brief_2023년12월호.pdf 파일을 요약해주세요. 

[ASSISTANT]
PDF 파일에서 텍스트를 추출할 수 없거나 내용이 읽을 수 없는 형식으로 되어 있는 것으로 보입니다. 이러한 경우에는 PDF 파일을 직접 여는 접근이 필요할 수 있습니다.

대안을 위해, 만약 PDF의 내용을 요약하고자 하신다면, PDF 파일을 통해 특정 페이지를 선택하여 그 내용을 붙여넣어 주시면 그 내용을 바탕으로 요약할 수 있습니다. 혹은 PDF 파일의 다른 버전을 제공하셔도 좋습니다. 이렇게 하면 필요한 정보를 보다 정확히 제공할 수 있습니다. 추가 지침이나 정보를 기다리고 있습니다.



아래는 이전에 도출한 답변을 얻기까지 Assistant 가 수행한 단계를 출력한 결과입니다.

단계

1. `tool_calls`: 먼저, `retrieval` 도구를 사용하여 파일에서 정보를 검색합니다.
2. `message_createion`: 그런다음, 검색된 정보를 사용하여 답변을 생성합니다.


In [13]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread_id, run_id=run.id, order="asc"
)

for step in run_steps.data:
    # 각 단계의 세부 정보를 가져옵니다.
    step_details = step.step_details
    # 세부 정보를 JSON 형식으로 출력합니다.
    show_json(step_details)

{'tool_calls': [{'id': 'call_9RDp0JsLpWoV1UYRxu00cDSS',
   'code_interpreter': {'input': "# SPRI_AI_Brief_2023년12월호.pdf 파일을 읽어들여 요약하기 위해\r\n# 파일 이름 확인 후 해당 파일 텍스트를 재추출 시도\r\nfile_path_pdf = '/mnt/data/file-AQULGHjWks1kQK2njsbAwjYF'\r\n\r\n# PDF 파일 읽어들이기\r\ntext_content_pdf = extract_text_from_pdf(file_path_pdf)\r\n\r\n# 처음 1000자 확인\r\ntext_content_pdf[:1000]",
    'outputs': []},
   'type': 'code_interpreter'}],
 'type': 'tool_calls'}

{'message_creation': {'message_id': 'msg_mPiQEdlkIw31dj8kXPmi3GQu'},
 'type': 'message_creation'}

In [15]:
run = ask(
    ASSISTANT_ID,
    thread_id,
    "Language Models are Unsupervised Multitask Learners 논문에서 가장 크게 기여한 연구 성과는 무엇인가요? "
    "200~300 단어로 자세히 한글로 설명해 주세요. 단, 기술적인 용어는 번역하지 마세요.",
)

[USER]
Language Models are Unsupervised Multitask Learners 논문에서 가장 크게 기여한 연구 성과는 무엇인가요? 200~300 단어로 자세히 한글로 설명해 주세요. 단, 기술적인 용어는 번역하지 마세요.

[ASSISTANT]
현재 저에게 업로드된 파일 목록에서는 "Language Models are Unsupervised Multitask Learners" 논문에 대한 파일이 존재하지 않습니다. 하지만 해당 논문의 주요 기여를 요약하여 설명할 수 있습니다.

"Language Models are Unsupervised Multitask Learners" 논문은 GPT (Generative Pre-trained Transformer) 모델의 기초가 되는 연구로, 자연어 처리(NLP) 분야에서의 혁신적인 기여로 평가됩니다. 이 연구의 가장 큰 성과는 대규모 언어 모델이 다양한 언어 처리 작업에서 뛰어난 성능을 발휘할 수 있다는 것을 입증한 것입니다. 

논문에서는 대규모 데이터셋에 대한 비지도 학습을 통해 모델이 다양한 작업에 대한 지식을 학습할 수 있음을 보여주었습니다. 특히, 이러한 모델이 여러 작업에서의 미세 조정(fine-tuning 없이) 수행하도록 훈련되었음에도 불구하고, 자연어 이해(NLU) 및 생성(NLG) 작업에서 우수한 성능을 나타내는 것을 강조했습니다. 

또한, 이 연구는 사전 학습된 언어 모델이 간단한 작업에서부터 복잡한 작업까지 하나의 모델로 해결할 수 있는 가능성을 보여주며, 이는 Multi-task Learning의 효율성을 극대화했습니다. 결과적으로, 다양한 어플리케이션에서 높은 정확도와 성능을 달성할 수 있는 기초가 되었으며, 이는 후속 연구와 기술 발전에 큰 영향을 미쳤습니다. 

이를 통해 GPT와 같은 모델들이 이후 NLP 연구 및 응용 분야에서 중심적인 역할을 하게 됨을 알 수 있습니다.



끝으로, 검색에는 [Annotations](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages)와 같은 더 많은 복잡한 부분이 있으며, 이는 다른 쿡북에서 다고 함.


뭔가 이상하지만.. 다시 찾아봐야할듯 일단 다음 실습으로... 쉬는시간 타임..